# A Dungeon Game Using Together API

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import json
import gradio as gr
from together import Together                                                                                                                              

def load_env():
    _ = load_dotenv(find_dotenv())

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def get_together_api_key():
     load_env()
     together_api_key = os.getenv("TOGETHER_API_KEY")
     return together_api_key

/opt/anaconda3/envs/dungeon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["TOGETHER_API_KEY"] = "your_api_key"

client = Together(api_key=get_together_api_key()) 

## Creating a World

In [3]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

In [4]:
world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""

In [5]:
output = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt},
    ],
)

In [6]:
world_output = output.choices[0].message.content
print(world_output)

World Name: Behemothia  
World Description: In Behemothia, cities are built on the backs of colossal, ancient beasts that roam the land. Each beast has its own unique ecosystem, and the cities adapt to their movements and behaviors. The people live in harmony with these creatures, relying on them for protection and resources.


In [7]:
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip().replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:]).replace('World Description:', '').strip()
}

## Generating Kingdoms

In [8]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in.
Describe important leaders, cultures, and history of the kingdoms.
Describe the history of war and peace between these kingdoms.
These kingdoms fight with each other to acquire the following 8 items:
gun, rose, socks, pants, short, hat, web, and gold resources.

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""

In [9]:
output = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [10]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0].split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  
  kingdom_description = output.strip().split('\n')[1].split('Description: ')[1].strip()
  
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name'],
  }
  
  kingdoms[kingdom_name] = kingdom

world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: {kingdom["description"]}')

Created kingdom "Zephyria" in Behemothia
Created kingdom "Terranox" in Behemothia
Created kingdom "Aquarion" in Behemothia
Created kingdom "Behemothia" in Behemothia

Kingdom 1 Description: In Behemothia, cities are built on the backs of colossal, ancient beasts that roam the land. Each beast has its own unique ecosystem, and the cities adapt to their movements and behaviors. The people live in harmony with these creatures, relying on them for protection and resources.


## Generating Towns

In [11]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""

In [12]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    
    output = client.chat.completions.create(
      model="deepseek-ai/DeepSeek-V3",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)},
      ],
    )
    
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\n\n'):
        town_name = output.strip().split('\n')[0].split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1].split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name'],
        }
        
        towns[town_name] = town
    
    kingdom["towns"] = towns

In [13]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Zephyria...
- Skyhaven created
- Galeport created
- Featherfall created

Creating towns for kingdom: Terranox...
- Ironhold created
- Verdant Vale created
- Emberwatch created

Creating towns for kingdom: Aquarion...
- Tidehaven created
- Deepshade created
- Wavecrest created

Creating towns for kingdom: Behemothia...
- Shellhaven created
- Embercliff created
- Windspire created

Town 1 Description: Shellhaven is built on the back of a massive, turtle-like beast that moves slowly across vast plains. The town is known for its sturdy, dome-shaped homes made from the beast’s shed shells. A central market thrives on rare herbs and minerals found in the beast’s wake.


## Generating Non-Player Characters (NPC's)

In [14]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 1 character based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as its deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [15]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    
    output = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)},
        ],
        temperature=1,  # added to generate unique names
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    
    for output in npcs_output.split('\n\n'):
        try:
            npc_name = output.strip().split('\n')[0].split('Name: ')[1].strip()
        except:
            npc_name = "Aslan"
            
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name'],
        }
        
        npcs[npc_name] = npc
    
    town["npcs"] = npcs

In [16]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)


Creating characters for the town of: Skyhaven...
- "Kael Swiftwind" created

Creating characters for the town of: Galeport...
- "Kael Windstrider" created

Creating characters for the town of: Featherfall...
- "Lira Windwhisper" created

Creating characters for the town of: Ironhold...
- "Kael Ironshield" created

Creating characters for the town of: Verdant Vale...
- "Elara Mosscrest" created

Creating characters for the town of: Emberwatch...
- "Caelum Ashveil" created

Creating characters for the town of: Tidehaven...
- "Aslan" created
- "Behemothia" created
- "Aquarion" created
- "Tidehaven" created

Creating characters for the town of: Deepshade...
- "Lira Swiftleaf" created

Creating characters for the town of: Wavecrest...
- "Kael Draven" created

Creating characters for the town of: Shellhaven...
- "Kael Thornweaver" created

Creating characters for the town of: Embercliff...
- "Kael Emberforge" created

Creating characters for the town of: Windspire...
- "Kael Veyin" created


In [17]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, {kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Windspire, Behemothia:
Kael is a lean, sharp-eyed glider pilot in Windspire, with wind-tousled dark hair and a scar across his left cheek from a near-fatal flight accident. He crafts and repairs gliders, his hands always stained with oils and feathers. Deep down, he longs to prove himself and escape the shadow of his legendary father, who disappeared in the mist-shrouded peaks years ago. Kael’s pain lies in his fear of failure, but his desire to soar higher than anyone ever has drives him forward.


## Saving the World

In [ ]:
save_world(world, '../dungeon_game_using_together_api/TheWorld.json')

## A Few Helper Functions

In [ ]:
def get_game_state(inventory={}):
    world = load_world('../dungeon_game_using_together_api/TheWorld.json')
    kingdom = list(world['kingdoms'].values())[0]
    town = list(kingdom['towns'].values())[0]
    character = list(town['npcs'].values())[0]
    name = world['name']

    game_state = {
        "world": world['description'],
        "kingdom": kingdom['description'],
        "town": town['description'],
        "character": character['description'],
        "start": name,
        "inventory": inventory,
    }
    
    return game_state

In [20]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']
        
    system_prompt = """You are an AI Game master. Your job is to write what \
                       happens next in a player's adventure game. \
                       Instructions: \
                       You must on only write 1-3 sentences in response. \
                       Always write in second person present tense. \
                       Ex. (You look north and see...)"""
    
    world_info = f"""
                 World: {game_state['world']}
                 Kingdom: {game_state['kingdom']}
                 Town: {game_state['town']}
                 Your Character: {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info},
    ]

    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})
           
    messages.append({"role": "user", "content": message})
    
    client = Together(api_key=get_together_api_key())
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages,
    )
    
    result = model_output.choices[0].message.content
    
    return result

In [21]:
def start_game(main_loop, share=False):
    demo = gr.ChatInterface(
           main_loop,
           chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
           textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
           title="AI RPG",
           theme="soft",
           examples=["Look around", "Continue the story"],
           cache_examples=False,
           retry_btn="Retry",
           undo_btn="Undo",
           clear_btn="Clear",
           )
   
    demo.launch(share=share, server_name="0.0.0.0")

## Checking for Safety

In [22]:
everyone_content_policy = {
    "policy": """
    O1: Violence and Hate.
    Should not
    - Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics.
    - Use slurs based on personal characteristics.
    - Write gruesome / graphic violence.
    Can
    - Write tame fantasy violence.
    O2: Sexual Content.
    Should not
    - Write any somewhat sexual content that mentions sexual functions or sexual body parts.
    - Write any content that would be inappropriate for kids.
    Can
    - Write romantic content (including kissing, flirting, etc.)
    O3: Self-Harm.
    Should not
    - Encourage people to harm themselves.
    - Romanticize or condone self-harm or suicide.
    - Write story depicting suicide.
    O4: Profanity.
    Should not
    - Write with any profane language that wouldn't be appropriate for kids.
    Can
    - Write with kid appropriate language.
    O5: Regulated or Controlled Substances.
    Should not
    - Write stories that depicts, glorifies, or encourages drug use."""
}

def is_safe(message):
    # Build the prompt with embedded values
    prompt = f"""[INST] Task: Check if there is unsafe content in 'user' messages in conversations according to our safety policy with the below categories.
    
    <BEGIN UNSAFE CONTENT CATEGORIES>
    {everyone_content_policy['policy']}
    <END UNSAFE CONTENT CATEGORIES>
    
    <BEGIN CONVERSATION>
    user: {message}
    
    Provide your safety assessment for user in the above conversation:
    - First line must read 'safe' or 'unsafe'.
    - If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""
    
    client = Together(api_key=get_together_api_key())

    response = client.completions.create(
        model="Meta-Llama/LlamaGuard-2-8b",
        prompt=prompt,
    )
    
    result = response.choices[0].text
    
    return result.strip() == 'safe'

## Define Inventory Detector

In [23]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update \
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []} \
and nothing else.

Response must be in Valid JSON.
Don't add items that were already added in the inventory.

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, "change_amount": <CHANGE AMOUNT>},
        ...
    ]
}
"""

In [24]:
client = Together(api_key=get_together_api_key())

In [28]:
def detect_inventory_changes(game_state, output):
    
    inventory = game_state['inventory']
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f'Current Inventory: {str(inventory)}'},
        
        {"role": "user", "content": f'Recent Story: {output}'},
        {"role": "user", "content": 'Inventory Updates'},
    ]
    
    chat_completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        temperature=0.0,
        messages=messages,
    )
    
    response = chat_completion.choices[0].message.content.split("json\n")[-1].rstrip("```")
    result = json.loads(response)
    return result['itemUpdates']

In [29]:
game_state = get_game_state()

game_state['inventory'] = {
    "gun": 1,
    "rose": 2,
    "socks": 3,
    "pants": 4,
    "short": 5,
    "hat": 6,
    "web": 7,
    "gold": 8,
}

result = detect_inventory_changes(game_state, "You buy a gun from the merchant for 5 gold.")

print(result)

[{'name': 'gun', 'change_amount': 1}, {'name': 'gold', 'change_amount': -5}]


In [30]:
def update_inventory(inventory, item_updates):
    update_msg = ''
    
    for update in item_updates:
        name = update['name']
        change_amount = update['change_amount']
        
        if change_amount > 0:
            if name not in inventory:
                inventory[name] = change_amount
            else:
                inventory[name] += change_amount
            
            update_msg += f'\nInventory: {name} + {change_amount}'
        
        elif name in inventory and change_amount < 0:
            inventory[name] += change_amount
            update_msg += f'\nInventory: {name} {change_amount}'
            
        if name in inventory and inventory[name] < 0:
            del inventory[name]
            
    return update_msg

#### Now include inventory in the story

In [31]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']
        
    system_prompt = """You are an AI Game master. Your job is to write what \
                       happens next in a player's adventure game. \
                       Instructions: \
                       You must on only write 1-3 sentences in response. \
                       Always write in second person present tense. \
                       Ex. (You look north and see...). \
                       Don't let the player use items they don't have in their inventory.
                    """

    world_info = f"""
                 World: {game_state['world']}
                 Kingdom: {game_state['kingdom']}
                 Town: {game_state['town']}
                 Your Character: {game_state['character']}
                 Inventory: {json.dumps(game_state['inventory'])}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info},
    ]

    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})
           
    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=messages,
    )
    
    result = model_output.choices[0].message.content
    
    return result

## Integrate into the Game

In [32]:
game_state = get_game_state(inventory={
    "gun": 1,
    "rose": 2,
    "socks": 3,
    "pants": 4,
    "short": 5,
    "hat": 6,
    "web": 7,
    "gold": 8,
})

In [33]:
def main_loop(message, history):
    output = run_action(message, history, game_state)
    
    safe = is_safe(output)
    if not safe:
        return 'Invalid Output'

    item_updates = detect_inventory_changes(game_state, output)
    
    update_msg = update_inventory(
        game_state['inventory'], 
        item_updates,
    )
    
    output += update_msg

    return output

start_game(main_loop, True)

Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://14cf4f84cbc94bd58e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
